In [1]:
import sys
import numpy as np
import pickle as pkl
from IPython.display import display
from maraboupy import Marabou, MarabouCore
from utils import TOTUtils

In [2]:
sample_input = [
    0.2622614500364243, -0.22479658685112885, -0.14872559153186365,-0.1183740939370412,       -0.11582643087693376, 0.4638055359970424, 0.5343624908151317, -1.8587238209603285,        -1.925346908225373, 0.7745258391529399, 1.691414092773877, -1.1513317010900714,           0.15687995017838718, -0.4932713694542802, 0.3758603164252629, -0.6443093237643779,        0.5198151301903109, -0.1392841543884293, -0.17885723928180608, -0.7026510235427964,       -0.04793722518544871, -0.4594806526602232, 0.18987588687321624, 0.4657886911920623,       -0.1626985247701109]
sample_output = [1, 0, 0]
samples = [(sample_input, sample_output)]

nnet_path = '../artifacts/models/model-v3.1.3/model.nnet'

In [3]:
# Max UNSAT distances per feature for sample_input
# distances = [0.0] * len(sample_input)
# distances[0] = 1.16
# distances[1] = 0.37
# distances[2] = 0.18
# distances[3] = 1.48
# distances[4] = 0.31
# distances[5] = 0.92
# distances[6] = 0.27
# distances[7] = 1.89
# distances[8] = 1.61
# distances[9] = 6.85
# distances[10] = 25.53
# distances[11] = 0.40
# distances[12] = 1.09
# distances[13] = 0.22
# distances[14] = 1.21
# distances[15] = 1.32
# distances[16] = 0.66
# distances[17] = 0.39
# distances[18] = 1.04
# distances[19] = 0.17
# distances[20] = 6.20
# distances[21] = 1.15
# distances[22] = 0.42
# distances[23] = 2.82
# distances[24] = 0.81

In [4]:
from robustness import evaluate_sample
x=0
d=1.17
s=sample_input[0:x] + [sample_input[x]+d] + sample_input[x+1:]
evaluate_sample(nnet_path, s, sample_output)

[['input 0 = 1.4322614500364241',
  'input 1 = -0.22479658685112885',
  'input 2 = -0.14872559153186365',
  'input 3 = -0.1183740939370412',
  'input 4 = -0.11582643087693376',
  'input 5 = 0.4638055359970424',
  'input 6 = 0.5343624908151317',
  'input 7 = -1.8587238209603285',
  'input 8 = -1.925346908225373',
  'input 9 = 0.7745258391529399',
  'input 10 = 1.691414092773877',
  'input 11 = -1.1513317010900714',
  'input 12 = 0.15687995017838718',
  'input 13 = -0.4932713694542802',
  'input 14 = 0.3758603164252629',
  'input 15 = -0.6443093237643779',
  'input 16 = 0.5198151301903109',
  'input 17 = -0.1392841543884293',
  'input 18 = -0.17885723928180608',
  'input 19 = -0.7026510235427964',
  'input 20 = -0.04793722518544871',
  'input 21 = -0.4594806526602232',
  'input 22 = 0.18987588687321624',
  'input 23 = 0.4657886911920623',
  'input 24 = -0.1626985247701109',
  'output 0 = 3.5931542051395406',
  'output 1 = 3.615030909085161',
  'output 2 = -4.356136826230461']]

In [5]:
# from robustness import find_feature_sensitivity_bruteforce
# find_feature_sensitivity_bruteforce(nnet_path, 0, samples)

from robustness import test_network_sensitivity
test_network_sensitivity(nnet_path, len(samples[0][0]), samples)

{'x0': (-4.93, 1.17),
 'x1': (-0.82, 0.38),
 'x2': (-0.32, 0.19),
 'x3': (-2.5, 1.49),
 'x4': (-0.32, 1.17),
 'x5': (-0.93, 0),
 'x6': (-1.08, 0.28),
 'x7': (-3.94, 1.9),
 'x8': (-1.62, 2.62),
 'x9': (-6.86, 9.24),
 'x10': (-40.16, 25.54),
 'x11': (-11.95, 0.41),
 'x12': (-1.58, 1.1),
 'x13': (-0.23, 4.1),
 'x14': (-1.95, 1.22),
 'x15': (-3.86, 1.33),
 'x16': (-0.67, 1.84),
 'x17': (-0.4, 0.66),
 'x18': (-1.05, 1.28),
 'x19': (-0.18, 0.5),
 'x20': (-0.16, 6.21),
 'x21': (-2.02, 1.16),
 'x22': (-0.43, 0.9),
 'x23': (-2.83, 8.19),
 'x24': (-0.82, 8.24)}